<a href="https://colab.research.google.com/github/hijyun/NLP/blob/main/seminar/SKT_OpenNMT_Pytorch_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenNMT Pytorch를 이용한 기계번역 모델 만들어보기

***First Go to Runtime and  change the runtime type to GPU.***


<br>
 Copyright Park Chanjun
<br>
 Email: bcj1210@naver.com










# Git Clone

In [ ]:
!git clone https://github.com/OpenNMT/OpenNMT-py            

fatal: destination path 'OpenNMT-py' already exists and is not an empty directory.


# Please install requirements.txt use by pip

> Error : You must restart the runtime in order to use newly installed versions.<br>
Solution : Click Restart Runtime => Redo



In [ ]:
!pip install -r OpenNMT-py/requirements.txt


  Cloning https://github.com/pytorch/text.git (to revision master) to /tmp/pip-req-build-exolrftf
  Running command git clone -q https://github.com/pytorch/text.git /tmp/pip-req-build-exolrftf
  Created wheel for torchtext: filename=torchtext-0.4.0-cp36-none-any.whl size=53405 sha256=8162594b1ceb45bdb2ca4eeeea7dd61fc2ff9dabecdee9099ac194a25a6c8c04
  Stored in directory: /tmp/pip-ephem-wheel-cache-tk79l7ub/wheels/47/f9/8d/a9e397ec2629a3fd3219b2ebc3ec8b55396fd3cf55963a77a5
Successfully built torchtext


# 기계번역 모델 만드는 순서






**1.   데이터 수집**

병렬 코퍼스를 수집해야합니다.
이번 실습에서는 OpenNMT에서 제공하는 WMT 데이터를 사용합니다.

**2.   정제, 병렬 코퍼스 필터링**

병렬 코퍼스 필터링 (생략)


**3. 서브워드 분리**

BPE를 사용


**4. 학습**

GPU를 이용한 Transformer 훈련


**5. 번역**

모델을 이용한 번역


**6. Detokenization**

디토큰


**7. 점수계산**

BLEU 점수 계산

# Subword Tokenization

We use Byte Pair Encoding for Subword Tokenization

https://www.aclweb.org/anthology/P16-1162

i => input<br>
o ==> Output(*.code)<br>
s ==> Symbol<br>

learn_bpe ==> make code<br>
apply_bpe ==> apply subwordTokenization<br>

src-train, src-val,test ==> Need to apply src.code<br>
tgt-train,tgt-val ==> Need to apply tgt.code

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-train.txt -o OpenNMT-py/data/src-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-val.txt -o OpenNMT-py/data/src-val-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/src.code -i OpenNMT-py/data/src-test.txt -o OpenNMT-py/data/src-test-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-train.txt -o OpenNMT-py/data/tgt-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c OpenNMT-py/data/tgt.code -i OpenNMT-py/data/tgt-val.txt -o OpenNMT-py/data/tgt-val-bpe.txt

# **Preprocess the data**

We will be working with some example data in data/ folder.

The data consists of parallel source (src) and target (tgt) data containing one sentence per line with tokens separated by a space:

1. src-train.txt

2. tgt-train.txt

3. src-val.txt

4. tgt-val.txt


Train data and validataion data are required for machine translation training.

Validation files are required and used to evaluate the convergence of the training. It usually contains no more than 5000 sentences.


> If you think about it briefly, you can specify the path of train data and validation data, and specify the path and name to save in -save_data.

> If you want to set vocab size add below command
<br>
-src_vocab_size 32000 -tgt_vocab_size 32000

The vocab size is usually 32000.

In [ ]:
!python OpenNMT-py/preprocess.py -train_src OpenNMT-py/data/src-train-bpe.txt -train_tgt OpenNMT-py/data/tgt-train-bpe.txt -valid_src OpenNMT-py/data/src-val-bpe.txt -valid_tgt OpenNMT-py/data/tgt-val-bpe.txt -save_data OpenNMT-py/data/demo -src_vocab_size 32000 -tgt_vocab_size 32000

[2019-08-28 07:39:58,435 INFO] Extracting features...
[2019-08-28 07:39:58,436 INFO]  * number of source features: 0.
[2019-08-28 07:39:58,436 INFO]  * number of target features: 0.
[2019-08-28 07:39:58,436 INFO] Building `Fields` object...
[2019-08-28 07:39:58,436 INFO] Building & saving training data...
[2019-08-28 07:39:58,436 INFO] Reading source and target files: OpenNMT-py/data/src-train-bpe.txt OpenNMT-py/data/tgt-train-bpe.txt.
[2019-08-28 07:39:58,441 INFO] Building shard 0.
[2019-08-28 07:39:58,802 INFO]  * saving 0th train data shard to OpenNMT-py/data/demo.train.0.pt.
[2019-08-28 07:39:59,246 INFO]  * tgt vocab size: 10034.
[2019-08-28 07:39:59,263 INFO]  * src vocab size: 9739.
[2019-08-28 07:39:59,316 INFO] Building & saving validation data...
[2019-08-28 07:39:59,316 INFO] Reading source and target files: OpenNMT-py/data/src-val-bpe.txt OpenNMT-py/data/tgt-val-bpe.txt.
[2019-08-28 07:39:59,318 INFO] Building shard 0.
[2019-08-28 07:39:59,373 INFO]  * saving 0th valid dat

# **Train the data(Transformer)**

https://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf


> If you get GPU-related errors, try halving batch_size

**Below is the full command, and if you want to know more about it, search about Transformer.**

!python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model OpenNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  

In [ ]:
!nvidia-smi

Wed Aug 28 07:40:00 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
!python OpenNMT-py/train.py -data OpenNMT-py/data/demo -save_model Op
enNMT-py/data/model/model -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 -encoder_type transformer -decoder_type transformer -position_encoding -train_steps 200000 -max_generator_batches 2 -dropout 0.1 -batch_size 4096 -batch_type tokens -normalization tokens -accum_count 2 -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 -param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 -world_size 1 -gpu_rank 0  

[2019-08-28 07:40:03,078 INFO]  * src vocab size = 9739
[2019-08-28 07:40:03,078 INFO]  * tgt vocab size = 10034
[2019-08-28 07:40:03,078 INFO] Building model...
[2019-08-28 07:40:10,009 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(9739, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax()
          (dropout): Dropout(p=0.1)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
       

# **Translate**

Now that you have your model, you can start translating.

-model ==> Setting your model

Output predictions into pred.txt

In [ ]:
!python OpenNMT-py/translate.py -model OpenNMT-py/data/model/model_step_1000.pt -src OpenNMT-py/data/src-val.txt -output OpenNMT-py/data/pred.txt -replace_unk -verbose -gpu 0

[2019-08-28 08:14:34,217 INFO] Translating shard 0.

SENT 1: ['Parliament', 'Does', 'Not', 'Support', 'Amendment', 'Freeing', 'Tymoshenko']
PRED 1: Das ist es nicht , dass Sie die Kommission .
PRED SCORE: -23.2571

SENT 2: ['Today', ',', 'the', 'Ukraine', 'parliament', 'dismissed', ',', 'within', 'the', 'Code', 'of', 'Criminal', 'Procedure', 'amendment', ',', 'the', 'motion', 'to', 'revoke', 'an', 'article', 'based', 'on', 'which', 'the', 'opposition', 'leader', ',', 'Yulia', 'Tymoshenko', ',', 'was', 'sentenced', '.']
PRED 2: Es ist , dass die Europäische Union in der Kommission in der Kommission in der Union .
PRED SCORE: -31.8308

SENT 3: ['The', 'amendment', 'that', 'would', 'lead', 'to', 'freeing', 'the', 'imprisoned', 'former', 'Prime', 'Minister', 'was', 'revoked', 'during', 'second', 'reading', 'of', 'the', 'proposal', 'for', 'mitigation', 'of', 'sentences', 'for', 'economic', 'offences', '.']
PRED 3: Das ist es , dass die Europäische Union in der Kommission in der Kommission i

# Detokenization

Even after the translation process is finished, it is still in a segment, so it is different from the actual sentence structure used by real people. Thus, when you perform a detoxification process, it is returned in the form of the actual sentence.

We Use "sed" for BPE Detokenization


In [ ]:
!sed -i "s/@@ //g"  OpenNMT-py/data/pred.txt

# Evaluation Using BLEU

Quantitative evaluation is performed on the sentence thus obtained. BLEU is a quantitative evaluation method for machine translation. You can see which model is superior by comparing it to the BLEU score you are comparing.

https://www.aclweb.org/anthology/P02-1040

In [ ]:
!perl  OpenNMT-py/tools/multi-bleu.perl OpenNMT-py/data/tgt-val.txt < OpenNMT-py/data/pred.txt

BLEU = 0.35, 25.3/2.0/0.3/0.0 (BP=0.538, ratio=0.617, hyp_len=44217, ref_len=71666)


If you have Any Question Please Email to  "bcj1210@naver.com"